In [4]:
import pandas as pd
import numpy as np
from gensim.models.wrappers.dtmmodel import DtmModel
from gird import GirdID2LngLat
import ast

In [22]:
model_filepath = "C:\\code\\topic modeling dtm\\src\\model\\"
model_filename = 'model-stayFalse20140803_sequence_full'
ldaseq = DtmModel.load(model_filepath + model_filename)

In [23]:
# print topic information
print(type(ldaseq))
print([item for item in dir(ldaseq)])

<class 'gensim.models.wrappers.dtmmodel.DtmModel'>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__ignoreds', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__numpys', '__recursive_saveloads', '__reduce__', '__reduce_ex__', '__repr__', '__scipys', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'alpha', 'convert_input', 'dtm_coherence', 'dtm_path', 'dtm_vis', 'em_steps', 'fcorpus', 'fcorpustxt', 'fem_steps', 'finit_alpha', 'finit_beta', 'flda_ss', 'fout_gamma', 'fout_influence', 'fout_liklihoods', 'fout_observations', 'fout_prob', 'foutname', 'ftimeslices', 'gamma_', 'id2word', 'influences_time', 'init_alpha', 'init_beta', 'init_ss', 'initialize_lda', 'lambda_', 'lda_max_em_iter', 'lda_sequence_max_iter', 'lda_sequence_min_iter', 'lencorpus', 'lhood_', 'lo

top_chain_var: 超参数

In [24]:
dictionary = ldaseq.id2word #dictionary
num_terms = ldaseq.num_terms # The number of features (terms) in corpus.
num_topics = ldaseq.num_topics 
lencorpus = ldaseq.lencorpus
time_slices = ldaseq.time_slices # time id
# 获取主题状态 主题个数，时间区间等基本信息
def getTopicModelInfo(num_terms=num_terms, num_topics=num_topics, lencorpus=lencorpus, time_slices=time_slices):
    dic = {
        'topic_number': num_topics,
        'topic_index_id': [i for i,v in enumerate(range(num_topics))],
        'time_rawdata_id': time_slices,
        'time_index_id': [i for i,v in enumerate(time_slices)],
        'time_range': [i+6 for i,v in enumerate(time_slices)],
        'corpus_features': num_terms,
        'corpus_len': lencorpus
    }
    return dic
topicModelInfo = getTopicModelInfo()

In [28]:
_lentime = 18
_lentopic = 18
#topicid range (0, 18); time_slice range(0, 18) indicate-> (6, 24)
# whole day
wholeday = {'info': "one day topics with tree hierarchy like: '{hour-topic-entity}' "}
for i, item_time in enumerate(range(_lentime)):
    print(item_time)
    wholeday.update({item_time: {}})
    for j, item_topic in enumerate(range(_lentopic)):
        topic = ldaseq.show_topic(topicid=item_topic, time=item_time, topn=10)
        concate = np.concatenate(
                (np.array(topic), 
                    np.array([v.split('-') for v in np.array(np.array(topic))[:,1]])
                ),
            axis=1)
        concate = np.delete(concate, 1, axis=1)
        wholeday[item_time].update({item_topic: concate})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [32]:
range()

TypeError: 'list' object cannot be interpreted as an integer

In [5]:
wholeday

{'info': "one day topics with tree hierarchy like: '{hour-topic-entity}' ",
 0: {0: array([['0.012373265563398213', '29187', '29186'],
         ['0.011289541736063434', '28322', '28105'],
         ['0.009941444145347303', '29187', '29188'],
         ['0.009564243888618486', '28321', '28322'],
         ['0.009446633572499006', '28535', '28536'],
         ['0.009327541806607798', '29188', '29187'],
         ['0.008879633996246616', '28752', '28535'],
         ['0.008633111584690267', '28968', '28967'],
         ['0.008405432897364522', '28967', '28968'],
         ['0.008371498970484956', '28539', '28538']], dtype='<U32'),
  1: array([['0.010150806901253976', '26345', '26562'],
         ['0.009914072847165915', '26346', '26345'],
         ['0.009432445524835527', '26561', '26778'],
         ['0.009371349022570441', '26562', '26561'],
         ['0.007869747524388574', '25699', '25698'],
         ['0.007484485198731934', '25915', '25914'],
         ['0.007454181534161609', '26994', '26993']

In [9]:
ldaseq.show_topic(topicid=1, time=1, topn=10)

[(0.009344133192593553, '26345-26562'),
 (0.009204051176266022, '26346-26345'),
 (0.008739305002344017, '26562-26561'),
 (0.008707282482244953, '26561-26778'),
 (0.007277040857835567, '25699-25698'),
 (0.007101743332010828, '27861-28078'),
 (0.0069509091888959665, '28077-28076'),
 (0.006920177387728034, '25915-25914'),
 (0.006847730405959202, '26994-26993'),
 (0.006767304023014726, '26778-26777')]

In [12]:
# 主题演变 单个主题+多个时间
def topicEvolution(ldaseq, topicid=1, timeIdRange=range(0,4), topn=10):
    dic = {'info': "topic evolution", 
            'topicid': topicid,
            'timeIdRange': timeIdRange,
            'topn': topn,
            'data': []}
    
    for time, v in enumerate(timeIdRange):
        topic = ldaseq.show_topic(topicid=topicid, time=time, topn=10)
        concate = np.concatenate(
                (np.array(topic), 
                    np.array([v.split('-') for v in np.array(np.array(topic))[:,1]])
                ),
            axis=1)
        concate = np.delete(concate, 1, axis=1)
        dic['data'].append({'time': time, 'topic': concate})
    return dic

topicEvolution(ldaseq, topicid=1, timeIdRange=range(0,4), topn=10)

{'info': "topic generation: '{hour-topic-entity}' ",
 'topicid': 1,
 'timerange': range(6, 10),
 'topn': 10,
 'data': [{'time': 0,
   'topic': array([['0.010150806901253976', '26345', '26562'],
          ['0.009914072847165915', '26346', '26345'],
          ['0.009432445524835527', '26561', '26778'],
          ['0.009371349022570441', '26562', '26561'],
          ['0.007869747524388574', '25699', '25698'],
          ['0.007484485198731934', '25915', '25914'],
          ['0.007454181534161609', '26994', '26993'],
          ['0.007270546048347395', '26778', '26777'],
          ['0.007199945505846646', '27861', '28078'],
          ['0.007094429239521544', '25698', '25915']], dtype='<U32')},
  {'time': 1,
   'topic': array([['0.009344133192593553', '26345', '26562'],
          ['0.009204051176266022', '26346', '26345'],
          ['0.008739305002344017', '26562', '26561'],
          ['0.008707282482244953', '26561', '26778'],
          ['0.007277040857835567', '25699', '25698'],
          

In [18]:
# 主题分布 多个主题+单个时间
def timesliceTopic(ldaseq, topicIdList=[1,2,3], timeId=0, topn=10):
    dic = {'info': "timeslice topics", 
            'topicIdList': topicIdList,
            'timeId': timeId,
            'topn': topn,
            'data': []}
    
    for i, topicnum in enumerate(topicIdList):
        topic = ldaseq.show_topic(topicid=topicnum, time=timeId, topn=10)
        concate = np.concatenate(
                (np.array(topic), 
                    np.array([v.split('-') for v in np.array(np.array(topic))[:,1]])
                ),
            axis=1)
        concate = np.delete(concate, 1, axis=1)
        dic['data'].append({'timeId': timeId, 'topic': concate})
    return dic

timesliceTopic(ldaseq, topicIdList=[1,2,3], timeId=0, topn=10)

0 1
1 2
2 3


{'info': 'timeslice topics',
 'topicIdList': [1, 2, 3],
 'timeId': 0,
 'topn': 10,
 'data': [{'timeId': 0,
   'topic': array([['0.010150806901253976', '26345', '26562'],
          ['0.009914072847165915', '26346', '26345'],
          ['0.009432445524835527', '26561', '26778'],
          ['0.009371349022570441', '26562', '26561'],
          ['0.007869747524388574', '25699', '25698'],
          ['0.007484485198731934', '25915', '25914'],
          ['0.007454181534161609', '26994', '26993'],
          ['0.007270546048347395', '26778', '26777'],
          ['0.007199945505846646', '27861', '28078'],
          ['0.007094429239521544', '25698', '25915']], dtype='<U32')},
  {'timeId': 0,
   'topic': array([['0.018588734334511102', '30259', '30476'],
          ['0.018287958415595034', '29609', '29825'],
          ['0.017207409458091655', '30041', '30258'],
          ['0.016966340616339434', '30691', '30690'],
          ['0.01633072046685343', '30475', '30474'],
          ['0.016169566459944034'

In [ ]:
# 主题的轨迹顺序表示
# 轨迹树的淡化分叉表示
# dtm.exe使用什么抽样？

# DTM - 

In [19]:
filepath = '../assets/driver1_trajectory.csv'
data = pd.read_csv(filepath)
LAT_BIAS, LNG_BIAS = +0.0025, -0.0025
data['lat'], data['lng'] = data['lat'] + LAT_BIAS, data['lng'] + LNG_BIAS
resdata = {
    'columns': data.columns.tolist(),
    'data': data.values.tolist(),
    'initdata': 1
}

In [18]:
data['lat']+0.0025

0       30.656970
1       30.656970
2       30.656970
3       30.656970
4       30.656970
          ...    
2117    30.693475
2118    30.693476
2119    30.693478
2120    30.693478
2121    30.693480
Name: lat, Length: 2122, dtype: float64